# Limex Challenge - Quick Start Guide for Lime Trading API

This notebook demonstrates the **Lime Trading API**

For further information about Limex Challenge competition and full API documentation, please visit:  

🔗 [Limex Challenge](https://challenges.limex.com/)

🔗 [Lime Trading API Documentation](https://docs.lime.co/trader/)

🔗 [Limex Challenge Discord Group](https://discord.com/invite/DgBEgX5J)

🔗 [Limex Challenge Rules & FAQs](https://www.promo.limex.com/faqchallenges?utm_source=email&utm_medium=d_outreach&utm_campaign=promo)

In [ ]:
import requests
import json
import time
import pandas as pd

Input your credentials

In [ ]:
# Config - Replace with your credentials
CLIENT_ID = 'your_client_id'
CLIENT_SECRET = 'your_client_secret'
USERNAME = 'your_username'
PASSWORD = 'your_password'
ACCOUNT_NUMBER = "dmo-c001@demo" #your user ID + @demo, eg: dmo-c001 + @demo

# Endpoints
AUTH_URL = 'https://auth.lime.co/connect/token'
BALANCE_URL = 'https://api.lime.co/accounts'
ORDER_URL = 'https://api.lime.co/orders/place'
PRICE_HIST_URL = 'https://api.lime.co/marketdata/history'

Basic API functions

In [ ]:
# Get Access Token
def get_access_token():
    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/x-www-form-urlencoded'
    }

    data = {
        'grant_type': 'password',
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET,
        'username': USERNAME,
        'password': PASSWORD
    }

    response = requests.post(AUTH_URL, headers=headers, data=data)

    if response.status_code == 200:
        token_data = response.json()
        return token_data['access_token']
    else:
        raise Exception(f"Failed to get token: {response.status_code}, {response.text}")

# Fetch OHLC Data by ticker symbol
def fetch_price_data(symbol, access_token):
    headers = {
        'Accept': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }

    # Calculate UNIX timestamps for the last 3 days (needed for 'from' and 'to')
    now = int(time.time())
    from_time = now - (3 * 24 * 60 * 60)  # In this example - 3 days in seconds

    params = {
        'symbol': symbol,
        'period': 'minute_15',  # The supported periods are: minute, minute_5, minute_15, minute_30, hour, day, week, month, quarter, year
        'from': from_time,
        'to': now
    }

    response = requests.get(PRICE_HIST_URL, headers=headers, params=params)

    if response.status_code != 200:
        raise Exception(f"Failed to fetch price data: {response.status_code}, {response.text}")

    candles = response.json()

    # Convert candle data to a DataFrame for easy processing
    df = pd.DataFrame(candles)
    df.drop(columns=['period'], inplace=True) # clean up and show only OHLC data

    # Convert Unix timestamp to datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    df.set_index('timestamp', inplace=True)

    return df

# Place an Order
def place_order(access_token, order_payload):
    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }

    response = requests.post(ORDER_URL, headers=headers, json=order_payload)

    if response.status_code == 200:
        print("Order placed successfully:")
        print(response.json())
    else:
        raise Exception(f"Failed to place order: {response.status_code}, {response.text}")

# Get Account Balances
def get_account_balances(access_token):
    headers = {
        'Accept': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }

    response = requests.get(BALANCE_URL, headers=headers)

    if response.status_code == 200:
        accounts = response.json()
        print("Accounts and Balances:")
        print(json.dumps(accounts, indent=2))
        return accounts
    else:
        raise Exception(f"Failed to fetch balances: {response.status_code}, {response.text}")



Get Account Balances and check on Connectivity

In [ ]:
try:
    token = get_access_token()
    print(f"✅ Successfully connected to the Limex API server. \n\nAccess Token: {token[:15]}...")  # Quick check of first part of token

    get_account_balances(token)

except Exception as e:
    print("Error:", str(e))

Fetch Historical Stock data - 3 days OHLC price of `AAPL`

In [ ]:
data = fetch_price_data('AAPL', token)
data.tail(10)

Place a Test Order

- Once you have successfully placed the order, please check it on the demo account to ensure it has been processed correctly.

In [ ]:
# Fill in the Order Details below
order_payload = {
    "account_number": ACCOUNT_NUMBER,
    "symbol": "AAPL",
    "quantity": 1,
    #"price": 250.00, #price only for limit order
    "time_in_force": "day",
    "order_type": "market",
    "side": "buy",
    "exchange": "auto",
}

try:
    token = get_access_token()
    print(f"Access Token: {token[:15]}...")  # Quick check of first part of token

    # Place an order
    place_order(token, order_payload)

except Exception as e:
    print("Error:", str(e))